Info-284 Group exam
Group members: Heejung Yu, Tsz Ching, Sverre-Emil and Aaron Male

# Table of Contents
1. [Introduction](#Introduction)
2. [Preprocessing](#Preprocessing)
    - [Features](#Features)
        - [Species](#Species)
        - [Equipment](#Equipment)
        - [Gross Weight of Catch](#Gross-Weight-of-Catch)
        - [Boat Information](#Boat-Information)
        - [Location of Trip](#Location-of-Trip)
        - [Drag Distance](#Drag-Distance)
        - [Duration](#Duration)
        - [Time](#Time)
    - [Analyzation](#Analyzation)
        - [Hovedart FAO](#Hovedart-FAO)
        - [Lengdegruppe](#Lengdegruppe)
        - [Redskap FAO](#Redskap-FAO)
        - [Rundvekt](#Rundvekt)
3. [Supervised Learning](#Supervised-Learning)
    - [Picking the Machine Learning Models](#Picking-the-Machine-Learning-Models)
        - [K-NN](#k-NN)
        - [Linear Models](#Linear-Models)
        - [Naive Bayes](#Naive-Bayes)
        - [Decision Trees](#Decision-Trees)
        - [Ensembles of Decision Trees](#Ensembles-of-Decision-Trees)
        - [Neural Networks](#Neural-Networks)
    - [Our Choices](#Our-Choices)

## Introduction 
In this notebook we will be discussing our approach to the Norwegian Directorate of Fisheries’ dataset. 

Reading through the official documentations ("Samansette driftsdata for fangst (setel) og fartøy" and “datadokumentasjon-ers-rapport-varnivaa-5-140121”) provided alongside the dataset we noticed that the dataset is sourced from the directorates final/landing receipts and vessel register. The final and landing receipts are filled out (often manually) when the ships arrive at the harbor by both the ship representative and the receiver, these receipts are vulnerable to human error when filled. This vulnerability is what we have based our machine learning project on, so we decided to try to predict Bycatch. We define Bycatch as instances where the species caught isn´t the same as the main species. We are aware this definition may not be an ideal definition due to the way the main species is classified, however this was the only definition we could make with the data provided. We believe if our model can predict Bycatch, it can detect illegal or accidental misreporting. 

In [ ]:
import warnings # Got an irritating warning
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows", None)
pd.set_option("float_format", "{:f}".format)

## Preprocessing

### Overview
This dataset has 45 different features and 305 434 samples, it consists of floats, integers, and strings.  Many features are similar, so we categorized them into 8 groups.
<br>
<br>
The 8 categories are as follows:
<ul>
<li>Species</li>
<li>Equipments used</li>
<li>Gross weight of catch</li>
<li>Boat information</li>
<li>Location of trip</li>
<li>Drag distance</li>
<li>Duration</li>
<li>Time</li>
</ul>

#### Feature selection
We manually selected our features by plotting data from each feature into various plots. Based on these visualizations, we selected the features we assumed were relevant. We made two datasets: one for Bycatch and another for main species only, using these datasets we could gauge the impact of each feature on our model.

In [ ]:
dataset = pd.read_csv("H:\Informasjonsvitenskap\Programming\Python\Info-284\Info284_Project\Exam Task\Dataset\elektronisk-rapportering-ers-2018-fangstmelding-dca-simple.csv", sep = ";")

# Dataset where the species isn"t the same as the main-species
bycatch = dataset[dataset["Art FAO"] != dataset["Hovedart FAO"]]

# Dataset where the species is the same as the main-species
main_species = dataset[dataset["Art FAO"] == dataset["Hovedart FAO"]]

#### Species

This feature consisted of "Hovedart FAO" and "Art FAO", this feature is necessary as it´s how we identified Bycatch.

#### Equipments

To measure the relevance of equipment, we compared the distribution of equipment in the original dataset and the Bycatch dataset.

In [ ]:
# A count of instances of equipment used for every Bycatch. 
count_of_equipment_used_for_only_Bycatch = bycatch.groupby(["Redskap FAO"])["Redskap FAO"].count()
count_of_equipment_used_for_only_Bycatch = count_of_equipment_used_for_only_Bycatch.sort_values(ascending=False)
count_of_equipment_used_for_only_Bycatch.plot(kind="bar")

In [ ]:
# A count of instances of equipment used for every species. 
count_of_equipment_used_for_original_dataset = dataset.groupby(["Redskap FAO"])["Redskap FAO"].count()
count_of_equipment_used_for_original_dataset = count_of_equipment_used_for_original_dataset.sort_values(ascending=False)
count_of_equipment_used_for_original_dataset.plot(kind="bar")

We didn't see too much of a difference here. We then checked the correlation between the most common Bycatch species caught and the equipment used to catch them.

In [ ]:
# Finding the most common bycatch, defined by Bycatch species with highest roundweight. 
count_of_Bycatches_for_every_main_species = bycatch.groupby(["Art FAO"])["Rundvekt"].sum()

# Top 5 species 
top_5_common_bycatch = (count_of_Bycatches_for_every_main_species.sort_values(ascending=False))[:5]
top_5_common_bycatch = list(top_5_common_bycatch.index)

fig, ax = plt.subplots(figsize=(16, 12)) 

positions = np.arange(len(top_5_common_bycatch))*3  
width = 0.5

# Finding the most common equipment used for catching each of these species
for i, species in enumerate(top_5_common_bycatch):
    species_only_dataset = bycatch[bycatch["Art FAO"] == species]
    count_of_equipment_used = species_only_dataset.groupby("Redskap FAO")["Redskap FAO"].count()
    top_equipment_for_species = count_of_equipment_used.sort_values(ascending=False).head(5) 
    
    for j, equipment in enumerate(top_equipment_for_species.index):
        ax.bar(positions[i] + j*width, top_equipment_for_species[equipment], width, label=f"{species} - {equipment}")

ax.set_xlabel("Species and Equipment")
ax.set_ylabel("Count")
ax.set_title("Top Bycatch Species and Their Most Common Equipment")

ax.set_xticks(positions + width)
ax.set_xticklabels(top_5_common_bycatch)

plt.legend()
plt.xticks(rotation=45)
plt.show()

We noticed that there was a pattern for the equipment used. Trawls and nets appeared frequently for these fishes.

#### Gross weight of catch 

This feature was relevant because of how the main species is calculated: “Hoved art opp gitt i estimert vekt i kilo rund vekt. Art som det er fisket mest av i følge innrapporteringen” (datadokumentasjon-ers-rapport-varnivaa-5-140121, p. 11). If the entry has a low gross weight, it is more likely to be bycatch.

#### Boat information

When we checked the most common equipment for each boat size we saw equipment that was also common among Bycatches.

In [ ]:
every_lengthgroup = ["28 m og over", "21-27,99 m", "15-20,99 m"]

# Get an equipment count for each length group
for length_group in every_lengthgroup:
        lengdegruppe_dataset = dataset[dataset["Lengdegruppe"] == length_group]
        lengdegruppe_equipmentcount = lengdegruppe_dataset.groupby([lengdegruppe_dataset["Redskap FAO"]])["Redskap FAO"].count()
        lengdegruppe_equipmentcount.sort_values(ascending=False, inplace=True)
        print(f"Top 3 common equipment for boats in category: {length_group}\n{lengdegruppe_equipmentcount.head(3)}\n")

We see here that the distribution of equipment used in bigger boats is much more skewed towards trawls than the distribution in smaller boat. This led us to believe that trips made by bigger boats results in more Bycatch compared to smaller boats.

#### Location of trip

To check if the area was relevant for our prediction, we chose to compare two datasets where the species with the most samples (Torsk) was caught as Bycatch and main species.

In [ ]:
#  Dataset where the species was Torsk
torsk_only = dataset[dataset["Art FAO"] == "Torsk"]

#  Dataset where the Torsk was a Bycatch
torsk_only_Bycatch = torsk_only[torsk_only["Hovedart FAO"] != "Torsk"]

#  Dataset where the Torsk was the main species
torsk_only_main = torsk_only[torsk_only["Hovedart FAO"] == "Torsk"]

In [ ]:

# Count the occurrences of each unique value in "Hovedområde start", sort, and select top 10
top_10_areas_bycatch_start = torsk_only_Bycatch["Hovedområde start"].value_counts().head(10)

# Plot the top 10 areas
top_10_areas_bycatch_start.plot(kind="pie", figsize=(12, 10), fontsize=10, autopct="%1.1f%%", startangle=140)

plt.title("Top 10 Torsk Bycatch Main Start Areas", fontsize=12)
plt.ylabel("")
plt.legend(title="Main Start Areas", loc="upper right", bbox_to_anchor=(0, 1))
plt.show()


In [ ]:

# Count the occurrences of each unique value in "Hovedområde start", sort, and select top 10
top_10_areas = torsk_only_main["Hovedområde start"].value_counts().head(10)

# Plot the top 10 areas
top_10_areas.plot(kind="pie", figsize=(12, 10), fontsize=10, autopct="%1.1f%%", startangle=140)

plt.title("Top 10 Torsk Non-Bycatch Main Start Areas", fontsize=12)
plt.ylabel("")
plt.legend(title="Main Start Area", loc="upper right", bbox_to_anchor=(0, 1))
plt.show()

We noticed that the distribution of areas between the two datasets differed. 

#### Drag distance

We checked if drag distance was relevant by comparing values between Bycatch and the original. 


In [ ]:
print("Duration for original dataset:\n", dataset["Trekkavstand"].describe())
print("\n")
print("Duration for bycatch:\n",bycatch["Trekkavstand"].describe())

We didn’t notice anything interesting, we thought there might be a difference when comparing it with equipment.

In [ ]:
torsk_only_dragdistance = dataset[dataset["Art FAO"] == "Torsk"]
# Removing an outlier so its easier to read the graph
torsk_only_dragdistance = torsk_only_dragdistance[torsk_only_dragdistance["Trekkavstand"] < 200000]

torsk_as_Bycatch_dragdistance = torsk_only_dragdistance[torsk_only_dragdistance["Hovedart FAO"] != "Torsk"]
torsk_as_Bycatch_dragdistance = torsk_as_Bycatch_dragdistance[torsk_as_Bycatch_dragdistance["Redskap FAO"] == "Bunntrål, otter"]


torsk_as_main_dragdistance = torsk_only_dragdistance[torsk_only_dragdistance["Hovedart FAO"] == "Torsk"]
torsk_as_main_dragdistance = torsk_as_main_dragdistance[torsk_as_main_dragdistance["Redskap FAO"] == "Bunntrål, otter"]

In [ ]:
plt.figure(figsize=(10, 8))

# Scatter plot for bycatch
plt.scatter(torsk_as_Bycatch_dragdistance["Rundvekt"], torsk_as_Bycatch_dragdistance["Trekkavstand"], color="blue", alpha=0.5, label="Torsk as Bycatch")

# Adding labels and title
plt.xlabel("Gross Weight (kg)", fontsize=14)
plt.ylabel("Distance Traveled (km)", fontsize=14)
plt.title("Amount of Torsk Caught per Distance Traveled", fontsize=16)

# Adding a legend to distinguish between the two datasets
plt.legend(title="Catch Type", title_fontsize="13", fontsize="12")

# Show the plot
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))

# Scatter plot for main catch
plt.scatter(torsk_as_main_dragdistance["Rundvekt"], torsk_as_main_dragdistance["Trekkavstand"], color="red", alpha=0.5, label="Torsk as Main Catch")

# Adding labels and title
plt.xlabel("Gross Weight (kg)", fontsize=14)
plt.ylabel("Distance Traveled (km)", fontsize=14)
plt.title("Amount of Torsk Caught per Distance Traveled", fontsize=16)

# Adding a legend to distinguish between the two datasets
plt.legend(title="Catch Type", title_fontsize="13", fontsize="12")

# Show the plot
plt.show()

In [ ]:
# amount of entries from 40 000 to 75 000 in Bycatch
filtered_dragdistance = torsk_as_Bycatch_dragdistance[(torsk_as_Bycatch_dragdistance["Trekkavstand"] > 40000) & (torsk_as_Bycatch_dragdistance["Trekkavstand"] < 75000)]

total_instances = filtered_dragdistance.shape[0]

print(total_instances)

We concluded that there wasn´t enough difference between the two graphs, the distribution of distance traveled in both graphs were similar. 

#### Duration

In [ ]:
print("Duration for original dataset:\n", dataset["Varighet"].describe())
print("\n")
print("Duration for bycatch:\n", bycatch["Varighet"].describe())

We checked the distribution and compared the quartiles; they were nearly identical.

#### Time

We made a dataset focused on Bycatch and main species where the species was Torsk and the equipment used was "Bunntrål". This specific setup was chosen to ensure a fair comparison. We then analyzed the start times of fishing operations and visualized the distribution using a pie chart, which illustrated the variations in timing in a straightforward manner.

In [ ]:
# Dataset where the equipment used is "bunntrål"
torsk_only_Bycatch_bunntrål = torsk_only_Bycatch[torsk_only_Bycatch["Redskap FAO"] == "Bunntrål, otter"]
torsk_only_main_bunntrål = torsk_only_main[torsk_only_main["Redskap FAO"] == "Bunntrål, otter"]

# Convert columns to datetime format to extract the hour
torsk_only_Bycatch_bunntrål["Startklokkeslett"] = pd.to_datetime(torsk_only_Bycatch_bunntrål["Startklokkeslett"], format="%H:%M")
torsk_only_main_bunntrål["Startklokkeslett"] = pd.to_datetime(torsk_only_main_bunntrål["Startklokkeslett"], format="%H:%M")

# Extract the hour from the start time in both main and Bycatch dataframe
torsk_only_Bycatch_bunntrål["Startklokkeslett_time"] = torsk_only_Bycatch_bunntrål["Startklokkeslett"].dt.hour
torsk_only_main_bunntrål["Startklokkeslett_time"] = torsk_only_main_bunntrål["Startklokkeslett"].dt.hour

# counting hours
hourly_distribution_start = torsk_only_Bycatch_bunntrål["Startklokkeslett_time"].value_counts().sort_index()
hourly_distribution_start2 = torsk_only_main_bunntrål["Startklokkeslett_time"].value_counts().sort_index()

In [ ]:
# Make plot 
fig, ax = plt.subplots(figsize=(10, 8))
hourly_distribution_start.plot(kind="pie", autopct="%1.1f%%", startangle=90)
plt.title("Hourly Distribution of Torsk Bycatch Start Times")

plt.show()

In [ ]:
# Make plot 
fig, ax = plt.subplots(figsize=(10, 8))
hourly_distribution_start2.plot(kind="pie", autopct="%1.1f%%", startangle=90)
plt.title("Hourly Distribution of Torsk Bycatch Start Times")

plt.show()

We noticed there weren’t any noticeable difference between the two. 

In [ ]:
# Excluding all features that aren't relevant
dataset = dataset[["Hovedart FAO", "Art FAO", "Lengdegruppe", "Redskap FAO", "Rundvekt", "Hovedområde start"]]

We dropped all features except: “Art FAO” , “Hovedart FAO”, “Redskap FAO”, “Rundvekt”, “Lengdegruppe” , “Hovedområde start”. 

In hindsight, we could have benefited from automatic feature selection methods, such as univariate statistics (Müller & Guido, 2017, p. 236); however, we were not aware of this method at the time.

### Analyzation
In this section, we identified outliers and missing values in our dataset. We evaluated their significance, and decided whether to keep, modify, or remove them.

According to Müller and Guido (2017), outliers are odd data points that can lead to trouble for other scaling techniques. (p.133)

#### Hovedart FAO
These were our key features therefore; we didn’t remove or identify any outliers. We dropped all missing values in these features since without these features it is impossible to classify them as Bycatch.

In [ ]:
# Counting all Na values
print(dataset["Hovedart FAO"].isna().sum())
print(dataset["Art FAO"].isna().sum())

# Drop all rows where Hovedart FAO and Art FAO is NaN
dataset = dataset.dropna(subset=["Hovedart FAO", "Art FAO"])

print(dataset["Hovedart FAO"].isna().sum())
print(dataset["Art FAO"].isna().sum())

#### Lengdegruppe
There were only 3 categories in this feature, the only thing to look out for were missing values.

In [ ]:
# Counting all Na values
print(dataset["Lengdegruppe"].isna().sum())

lengdegruppe_is_NaN = dataset[(dataset["Lengdegruppe"].isna())]

# Checking samples where lengdegruppe is Na
pd.DataFrame(lengdegruppe_is_NaN.head(10))

There were only 683 missing values, we inspected them further to find any patterns. 

In [ ]:
# Printing the counts
print(lengdegruppe_is_NaN["Art FAO"].value_counts())

All the missing values were from catching kelp, we chose to convert them into its own category. 

In [ ]:
# Make new category in lengdegruppe
dataset["Lengdegruppe"] = dataset["Lengdegruppe"].fillna("Stortare båter")

#### Redskap FAO

In [ ]:
# Counting all Na values
print(dataset["Redskap FAO"].isna().sum())
redskap_FAO_is_NaN = dataset[dataset["Redskap FAO"].isna()]
pd.DataFrame(redskap_FAO_is_NaN).head(10)


Although there were 200 missing values in our dataset, the other features for these samples were complete. We determined that imputing or replacing these missing values would not significantly impact our analysis and would increase preprocessing work, we decided to drop them.

In [ ]:
# Drop Na values
dataset = dataset.dropna(subset=["Redskap FAO"])

#### Rundvekt
When analyzing outliers, we considered the varying average gross weights across different species. We used box plots for visualization because they effectively highlight outliers for each species.

In [ ]:
# Add all values in a list
species_grossweight = dataset.groupby("Art FAO")["Rundvekt"].apply(list)

# The three most popular fishes
species_of_interest = ["Torsk", "Hyse", "Sei"]

# Filter the data to include only the species we want
filtered_species_grossweight = {species: weights for species, weights in species_grossweight.items() if species in species_of_interest}

# Make box plots of gross weight
for species, grossweights in filtered_species_grossweight.items():
    plt.figure(figsize=(10, 6)) 
    plt.boxplot(grossweights)
    plt.title(f"Boxplot of Rundvekt for {species}")
    plt.ylabel("Rundvekt")
    plt.xticks([1], [species]) 
    
    plt.show()

Given the presence of many outliers across the 122 different species in our dataset, removing outliers proved difficult. Some species have very few, while others are significantly affected. According to Müller and Guido (2017), a practical approach is employing a RobustScaler. This method effectively transforms the data by minimizing the influence of extreme deviations, which is ideal for our dataset where outliers are common but cannot be simply removed. (p.133)

In [ ]:
# Checking for Na values
print(dataset["Rundvekt"].isna().sum())

#### Startområde

In [ ]:
# Checking for Na values
print(dataset["Hovedområde start"].isna().sum())

We found 4000 missing values; we dropped these samples.

In [ ]:
# Dropping Na values
dataset = dataset.dropna(subset=["Hovedområde start"])

## Supervised learning

### Picking the Machine learning models 
Considering our dataset has 5 features (4 categorical, 1 continuous) and we are working with a binary classification problem. We need to choose models that can effectively handle this data.

### Our choices 
- Logistic Regression: Logistic regression, described as the following according to Müller and Guido (2017), "Go-to as a first algorithm to try, good for very large datasets, good for very high-dimensional data" (p. 128), was found to be the best fit for our problem since it is one of the two most common linear classification algorithms (p. 56), and we have previous experience with it. This model will function as our baseline model due to its simplistic nature. 
  
- Gradient Boosting Forest: Gradient Boosting Forest, described as the following according to Müller and Guido (2017), "Often slightly more accurate than random forests. Slower to train but faster to predict than random forests, and smaller in memory. Need more parameter tuning than random forests" (p. 128), was chosen because it takes less memory than a random forest, which allows us to run it on most hardware

- Neural Network: explain sverre


### Transforming dataset

#### Target variable

In [ ]:
# Create a new binary feature to use as target variable. 
dataset["Is_Bycatch"] = (dataset["Hovedart FAO"] != dataset["Art FAO"])

#### Changing categorical data into numeric data
Our dataset includes 5 categorical features with numerous categories each, making One-hot Encoding impractical due to the high dimensionality it would introduce. Instead, we used cat.codes to convert these categories into numeric values. 

In [ ]:
# Changing categorical data into numeric data using "cat.codes"
dataset["Art FAO"] = dataset["Art FAO"].astype("category")
dataset["Art FAO Codes"] = dataset["Art FAO"].cat.codes

dataset["Lengdegruppe"] = dataset["Lengdegruppe"].astype("category")
dataset["Lengdegruppe Codes"] = dataset["Lengdegruppe"].cat.codes

dataset["Hovedområde start"] = dataset["Hovedområde start"].astype("category")
dataset["Hovedområde start Codes"] = dataset["Hovedområde start"].cat.codes 

dataset["Redskap FAO"] = dataset["Redskap FAO"].astype("category")
dataset["Redskap FAO Codes"] = dataset["Redskap FAO"].cat.codes 

# Excluding the old features
dataset = dataset[["Art FAO Codes", "Lengdegruppe Codes", "Redskap FAO Codes", "Rundvekt", "Hovedområde start Codes", "Is_Bycatch"]]

#### Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting dataset 
X = dataset[["Art FAO Codes", "Lengdegruppe Codes", "Redskap FAO Codes", "Rundvekt", "Hovedområde start Codes"]]
y = dataset["Is_Bycatch"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=50)

#### Scaling "Rundvekt"

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.compose import ColumnTransformer

# Scaling only "Rundvekt" and not the others
ct = ColumnTransformer(
    [("scale", RobustScaler(), ["Rundvekt"])],
    remainder="passthrough"
)

# Fitting the scaler to the training set and not the test set to prevent data leakage
X_train_scaled = ct.fit_transform(X_train)
X_test_scaled = ct.transform(X_test)

### Implementation and evaluation
We evaluated our model using cross-validation paired with a grid search to optimize performance:

Cross-Validation: "Cross-validation is a statistical method of evaluating generalization performance that is more stable and thorough than using a split into a training and a test set" (Müller & Guido, 2017, p. 252). We opted for the stratified k-Fold Cross-Validation, which was suitable for our dataset due to the significant disparities in sample sizes across different fish species. This method ensures each fold copies the overall dataset composition, thereby enhancing the reliability of our model evaluation.

Grid Search: This technique was employed to tune the model’s parameters, aiming to improve its generalization across new data sets.

Success Metric: The model's effectiveness was gauged by its ability to minimize false negatives, reducing potential misreporting within our data. This will be checked by utilizing a confusion matrix.

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import precision_recall_curve
from mglearn.datasets import make_blobs
from sklearn.pipeline import Pipeline

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Running a basic implementation of the model
logreg = LogisticRegression(random_state=10)

logreg.fit(X_train_scaled, y_train)

print("Accuracy on training set: {:.5f}".format(logreg.score(X_train_scaled, y_train)))
print("Accuracy on test set: {:.5f}".format(logreg.score(X_test_scaled, y_test)))

# Printing a confusion matrix to check false positives and negatives
cm = confusion_matrix(y_test, logreg.predict(X_test_scaled))
print("Confusion Matrix:")
print(cm)

The model’s accuracy on the training and test set were similar. The similarity in accuracy likely stems from the model's simplicity, which may not capture the underlying patterns, or from the data being too sparse for certain species.

Reviewing the confusion matrix, the false negative rate is low when compared to the false positives.  

To get a proper understanding of the accuracy of this model we will be running a stratified k-Fold before tuning the hyperparameter, since the dataset is large, we will be using 10 folds.

In [ ]:
# Running cross validation
logreg = LogisticRegression(random_state=10)

# Create StratifiedKFold object.
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)

# List to hold accuracies for train and test set
accuracy_cross_validation_test= []
accuracy_cross_validation_train = []
cnf_matrix = []

# for each train and test fold, scale it, fit it to the model append the accuracy score in their respective lists
for train_index, test_index in skf.split(X, y):

    # Makes a train_test split from the fold using indexing
    X_train_fold = X.iloc[train_index]
    X_test_fold = X.iloc[test_index]
    y_train_fold = y.iloc[train_index]
    y_test_fold = y.iloc[test_index]

    # Scale the X dataset
    X_train_fold_scaled = ct.fit_transform(X_train_fold)
    X_test_fold_scaled = ct.transform(X_test_fold)

    logreg.fit(X_train_fold_scaled, y_train_fold)

    accuracy_cross_validation_train.append(logreg.score(X_train_fold_scaled, y_train_fold))
    accuracy_cross_validation_test.append(logreg.score(X_test_fold_scaled, y_test_fold))
    cnf_matrix.append(confusion_matrix(y_test_fold, logreg.predict(X_test_fold_scaled)))

highest_diff = (0,0)
most_accurate_score = (0,0)
least_accurate_score = (0,1)

# Print out the accuracies
for i in enumerate(accuracy_cross_validation_test):
    train = accuracy_cross_validation_train[i[0]]
    test = accuracy_cross_validation_test[i[0]]
    diff = train-test

    # Finding interesting details
    if abs(diff) > abs(highest_diff[1]):
        highest_diff = (i[0], diff)
    
    if test > most_accurate_score[1]:
        most_accurate_score = (i[0], test)
    
    if test < least_accurate_score[1]:
        least_accurate_score = (i[0], test)

    print(f"Fold {i[0]+1}")
    print(f"Confusion Matrix:\n{cnf_matrix[i[0]]}")
    print(f"Accuracy score for the training set: {train:.3f}")
    print(f"Accuracy score for the test set: {test:.3f}") 
    print(f"Difference in test and train set: {diff}")
    print("\n")

print(f"The highest accuracy the model achieved was {most_accurate_score[1]}, this was in fold {most_accurate_score[0]+ 1 }")
print(f"The lowest accuracy the model achieved was {least_accurate_score[1]}, this was in fold {least_accurate_score[0]+ 1 }")
print(f"The highest difference the model got was {highest_diff[1]}, this was in fold {highest_diff[0] + 1}")

The model performs well, with a maximum train/test score difference of -0.004 and a minimum accuracy of 83.91%, indicating stability and consistency without overfitting. According to Müller and Guido (2017), "Overfitting occurs when you fit a model too closely to the particularities of the training set and obtain a model that works well on the training set but is not able to generalize to new data" (p. 28). The top accuracy suggests potential underfitting or issues with dataset imbalance according to Müller and Guido (2017), "Choosing too simple a model is called underfitting" (p. 28). This is likely due to large variances in sample sizes across species, making it easier to predict common species like 'torsk' compared to rarer ones like "steinbiter" or "lanternfish". 

Using a grid search, we can optimize our model's parameters to enhance its complexity and improve generalization. This should help in reducing false predictions in our dataset. Müller and Guido (2017) describes generalization as a model that is able to make accurate predictions on unseen data (p. 26). Parameters are changes you can make to the model to change how well it generalizes to the train data. We will use the F1 score to evaluate performance, according to Müller and Guido (2017), the F1 score takes precision and recall into account and can be a better measure on imbalanced binary classification datasets (p. 284).

The parameters adjusted will be:

C (Regularization Strength): Increasing C should help the model capture more data nuances due to a lower bias and higher variance.

Penalty (Regularization Norm): We will try the L2 norm and none. Because we consider all features important, we will not be trying L1 norm.

Solver: We will test “sag” and “newton-cholesky” against the default “lbfgs”. “Sag” is recommended for large datasets, and “newton-cholesky” is ideal for our case where the number of samples exceeds the number of features.

max_iter (Maximum Iterations): This will control the convergence of the solvers. Convergence refers to the process where the algorithm reaches a stable solution."

We built a pipeline for the grid search to prevent data leakage from the test split into the parameters. According to Müller and Guido (2017), pipelines prevents data leakage by refitting the robust scaler using only the training data for each cross-validation fold (p. 307).


In [ ]:
# Building a pipeline
pipe = Pipeline([("scaler", ct), ("logreg", LogisticRegression(random_state=10))])

# Defining our parameter grid
param_grid = {
    "logreg__C": [0.001, 0.01, 0.1, 1, 10,100],
    "logreg__penalty": ["None","l2"],
    "logreg__solver": ["lbfgs", "sag", "newton-cg"],
    "logreg__max_iter": [10, 100, 1000, 10000]}

# Using the same amount of k-Folds as the amount we used in cross validation
# Using an f1 scorer to score based on precision and not on accuracy
grid = GridSearchCV(pipe, param_grid=param_grid, cv=10, verbose=1, scoring=make_scorer(f1_score))

# Using the train test split from section [33]
grid.fit(X_train, y_train)

print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))
print("Best parameters: {}".format(grid.best_params_))

According to the gridsearch, the best parameters possible for our model is {'logreg__C': 100, 'logreg__max_iter': 10, 'logreg__penalty': 'l2', 'logreg__solver': 'sag'}. 

This combination achieved a very high accuracy. The “sag” solver was most efficient, likely due to large fold sizes (10,000+ samples). As anticipated, “L2” regularization was most effective given our handpicked features.

Surprisingly, a high C parameter value was better for the model, suggesting that less regularization (higher C implies weaker regularization) was beneficial. This indicates that our chosen features were distinct enough which made it easier to generalize.


In [ ]:
# Running cross validation with best parameters
logreg = LogisticRegression(C=100, max_iter=10, penalty="l2", solver="sag", random_state=10)
# Create StratifiedKFold object.
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)

# List to hold accuracies for train and test set
accuracy_cross_validation_test= []
accuracy_cross_validation_train = []
cnf_matrix = []

# for each train and test fold, scale it, fit it to the model append the accuracy score in their respective lists
for train_index, test_index in skf.split(X, y):

    # Makes a train_test split from the fold using indexing
    X_train_fold = X.iloc[train_index]
    X_test_fold = X.iloc[test_index]
    y_train_fold = y.iloc[train_index]
    y_test_fold = y.iloc[test_index]

    # Scale the X dataset
    X_train_fold_scaled = ct.fit_transform(X_train_fold)
    X_test_fold_scaled = ct.transform(X_test_fold)

    logreg.fit(X_train_fold_scaled, y_train_fold)

    accuracy_cross_validation_train.append(logreg.score(X_train_fold_scaled, y_train_fold))
    accuracy_cross_validation_test.append(logreg.score(X_test_fold_scaled, y_test_fold))
    cnf_matrix.append(confusion_matrix(y_test_fold, logreg.predict(X_test_fold_scaled)))

highest_diff = (0,0)
most_accurate_score = (0,0)
least_accurate_score = (0,1)

# Print out the accuracies
for i in enumerate(accuracy_cross_validation_test):
    train = accuracy_cross_validation_train[i[0]]
    test = accuracy_cross_validation_test[i[0]]
    diff = train-test

    # Finding interesting details
    if abs(diff) > abs(highest_diff[1]):
        highest_diff = (i[0], diff)
    
    if test > most_accurate_score[1]:
        most_accurate_score = (i[0], test)
    
    if test < least_accurate_score[1]:
        least_accurate_score = (i[0], test)

    print(f"Fold {i[0]+1}")
    print(f"Confusion Matrix:\n{cnf_matrix[i[0]]}")
    print(f"Accuracy score for the training set: {train:.3f}")
    print(f"Accuracy score for the test set: {test:.3f}") 
    print(f"Difference in test and train set: {diff}")
    print("\n")

print(f"The highest accuracy the model achieved was {most_accurate_score[1]}, this was in fold {most_accurate_score[0]+ 1 }")
print(f"The lowest accuracy the model achieved was {least_accurate_score[1]}, this was in fold {least_accurate_score[0]+ 1 }")
print(f"The highest difference the model got was {highest_diff[1]}, this was in fold {highest_diff[0] + 1}")

Adjusting parameters slightly increased model stability and accuracy, our model achieved a slightly higher peak accuracy

Results for Fold 6 showed a decrease in the false positive rate to 12.62% from 12.79% and an increase in the false negative rate to 2.76% from 2.60%, compared to the unoptimized model. This shift likely stems from the f1 scorer's emphasis on the harmonic mean of precision and recall, as discussed in "Introduction to Machine Learning" (p.283). 

To further reduce false negatives, we considered trading precision for recall. According to Müller and Guido (2017), a precision-recall curve can be used to see all possible trade-offs of precision and recall (p. 289). "Precision measures how many of the samples predicted as positive are actually positive" (p. 282). "Recall is used as a performance metric when we need to identify all positive samples; that is, when it is important to avoid false negatives" (p. 283).

In [ ]:

logreg = LogisticRegression(C=100, max_iter=10, penalty="l2", solver="sag", random_state=10).fit(X_train_scaled, y_train)
precision, recall, thresholds = precision_recall_curve(
y_test, logreg.decision_function(X_test_scaled))

# find threshold closest to zero
close_zero = np.argmin(np.abs(thresholds))
plt.plot(precision[close_zero], recall[close_zero], "o", markersize=10,
label="threshold zero", fillstyle="none", c="k", mew=2)
plt.plot(precision, recall, label="precision recall curve")
plt.xlabel("Precision")
plt.ylabel("Recall")

Ideally, we aim for the highest recall with minimal precision sacrifice, however, our model currently offers an acceptable balance between the two. Minor improvements in precision and recall wouldn't significantly enhance accuracy, so we are satisfied with the already low rate of false negatives.

### Gradient boosted regression trees

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
# Running a basic implementation
gbrt = GradientBoostingClassifier(random_state=10)

gbrt.fit(X_train_scaled, y_train)

print("Accuracy on training set: {:.3f}".format(gbrt.score(X_train_scaled, y_train)))
print("Accuracy on test set: {:.3f}".format(gbrt.score(X_test_scaled, y_test)))

# Printing a confusion matrix to check false positives and negatives
cm = confusion_matrix(y_test, gbrt.predict(X_test_scaled))
print("Confusion Matrix:")
print(cm)

The model got high accuracy on both the training and test sets, showing no signs of overfitting. The confusion matrix shows similar amount of false negatives and false positives. These low rates indicates a promising model.

In [ ]:
# Running a cross validation.
gbrt = GradientBoostingClassifier(random_state=10)

# Create StratifiedKFold object.
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)

# List to hold accuracies for train and test set
accuracy_cross_validation_test= []
accuracy_cross_validation_train = []
cnf_matrix = []

# for each train and test fold, scale it, fit it to the model append the accuracy score in their respective lists
for train_index, test_index in skf.split(X, y):

    # Makes a train_test split from the fold using indexing
    X_train_fold = X.iloc[train_index]
    X_test_fold = X.iloc[test_index]
    y_train_fold = y.iloc[train_index]
    y_test_fold = y.iloc[test_index]

    # Scale the X dataset
    X_train_fold_scaled = ct.fit_transform(X_train_fold)
    X_test_fold_scaled = ct.transform(X_test_fold)

    gbrt.fit(X_train_fold_scaled, y_train_fold)

    accuracy_cross_validation_train.append(gbrt.score(X_train_fold_scaled, y_train_fold))
    accuracy_cross_validation_test.append(gbrt.score(X_test_fold_scaled, y_test_fold))
    cnf_matrix.append(confusion_matrix(y_test_fold, gbrt.predict(X_test_fold_scaled)))

highest_diff = (0,0)
most_accurate_score = (0,0)
least_accurate_score = (0,1)

# Print out the accuracies
for i in enumerate(accuracy_cross_validation_test):
    train = accuracy_cross_validation_train[i[0]]
    test = accuracy_cross_validation_test[i[0]]
    diff = train-test

    # Finding interesting details
    if abs(diff) > abs(highest_diff[1]):
        highest_diff = (i[0], diff)
    
    if test > most_accurate_score[1]:
        most_accurate_score = (i[0], test)
    
    if test < least_accurate_score[1]:
        least_accurate_score = (i[0], test)

    print(f"Fold {i[0]+1}")
    print(f"Confusion Matrix:\n{cnf_matrix[i[0]]}")
    print(f"Accuracy score for the training set: {train:.3f}")
    print(f"Accuracy score for the test set: {test:.3f}") 
    print(f"Difference in test and train set: {diff}")
    print("\n")

print(f"The highest accuracy the model achieved was {most_accurate_score[1]}, this was in fold {most_accurate_score[0]+ 1 }")
print(f"The lowest accuracy the model achieved was {least_accurate_score[1]}, this was in fold {least_accurate_score[0]+ 1 }")
print(f"The highest difference the model got was {highest_diff[1]}, this was in fold {highest_diff[0] + 1}")

The model performs consistently well, it shows stable results across all folds. This model is achieving a 5% higher accuracy than our baseline model, which is very good considering it hasn't been tuned. There also seems so be no clear sign of underfitting or overfitting with the highest difference between test/train set being a 0.3%. The model may be performing better than our baseline model due to the complex relationships in our data. 

We ran a grid search to try to optimize the model. 

The parameters adjusted will be:
- max_depth: This parameter adjusts the depth limit in the tree, controlling the complexity of the decision tree model (scikit-learn, n.d.). According to Müller and Guido (2017), "max_depth is set very low for gradient boosted models, often not deeper than five splits." (p. 92). The default value is 3, we tried 2 and 5 to see if these were any good.

- n_estimators: According to Müller and Guido (2017), this parameter represents the number of trees in the model. Increasing the number of estimators can enhance model complexity and performance. The default value is 100, we tried larger numbers since it results in better performance (scikit-learn, n.d.).

- learning_rate: Müller and Guido (2017) explains that the learning rate determines the contribution of each tree in the ensemble. A higher learning rate allows each tree to make stronger corrections to the model (p. 89).

In [ ]:
pipe = Pipeline([("scaler", ct), ("gbrt", GradientBoostingClassifier(random_state=10))])

# Defining our parameter grid
param_grid = {
    "gbrt__max_depth": [2, 3, 5],  # Not deeper than five according to Müller and Guido (2017), ( p.92)
    "gbrt__n_estimators": [25, 50, 100, 200, 300, 400],  
    "gbrt__learning_rate": [0.01, 0.1, 0.2, 0.5,1]  
}


# Using the same amount of k-Folds as the amount we used in cross validation
# Using an f1 scorer to score based on precision and not on accuracy
grid = GridSearchCV(pipe, param_grid=param_grid, cv=10, verbose=1, scoring=make_scorer(f1_score))

# Using the train test split from section [33]
grid.fit(X_train, y_train)

print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))
print("Best parameters: {}".format(grid.best_params_))

We achieved a high F1 score using these parameters, suggesting that our model performs best with a high number of trees. This preference is likely due to the large number of samples in our dataset. The maximum depth of 5 might be optimal because it finds the complex interactions within our data.

In [ ]:
gbrt = GradientBoostingClassifier(learning_rate=0.2, max_depth=5 ,n_estimators=400, random_state=10)

# Create StratifiedKFold object.
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)

# List to hold accuracies for train and test set
accuracy_cross_validation_test= []
accuracy_cross_validation_train = []
cnf_matrix = []

# for each train and test fold, scale it, fit it to the model append the accuracy score in their respective lists
for train_index, test_index in skf.split(X, y):

    # Makes a train_test split from the fold using indexing
    X_train_fold = X.iloc[train_index]
    X_test_fold = X.iloc[test_index]
    y_train_fold = y.iloc[train_index]
    y_test_fold = y.iloc[test_index]

    # Scale the X dataset
    X_train_fold_scaled = ct.fit_transform(X_train_fold)
    X_test_fold_scaled = ct.transform(X_test_fold)

    gbrt.fit(X_train_fold_scaled, y_train_fold)

    accuracy_cross_validation_train.append(gbrt.score(X_train_fold_scaled, y_train_fold))
    accuracy_cross_validation_test.append(gbrt.score(X_test_fold_scaled, y_test_fold))
    cnf_matrix.append(confusion_matrix(y_test_fold, gbrt.predict(X_test_fold_scaled)))

highest_diff = (0,0)
most_accurate_score = (0,0)
least_accurate_score = (0,1)

# Print out the accuracies
for i in enumerate(accuracy_cross_validation_test):
    train = accuracy_cross_validation_train[i[0]]
    test = accuracy_cross_validation_test[i[0]]
    diff = train-test

    # Finding interesting details
    if abs(diff) > abs(highest_diff[1]):
        highest_diff = (i[0], diff)
    
    if test > most_accurate_score[1]:
        most_accurate_score = (i[0], test)
    
    if test < least_accurate_score[1]:
        least_accurate_score = (i[0], test)

    print(f"Fold {i[0]+1}")
    print(f"Confusion Matrix:\n{cnf_matrix[i[0]]}")
    print(f"Accuracy score for the training set: {train:.3f}")
    print(f"Accuracy score for the test set: {test:.3f}") 
    print(f"Difference in test and train set: {diff}")
    print("\n")

print(f"The highest accuracy the model achieved was {most_accurate_score[1]}, this was in fold {most_accurate_score[0]+ 1 }")
print(f"The lowest accuracy the model achieved was {least_accurate_score[1]}, this was in fold {least_accurate_score[0]+ 1 }")
print(f"The highest difference the model got was {highest_diff[1]}, this was in fold {highest_diff[0] + 1}")

We achieved a 2% increase in accuracy with the optimized model . This model also achieved a significantly higher recall rate, which is likely due to our choice of the F1 score as the success metric. Precision was generally improved as well.

However, there were indications of potential overfitting, as observed in the results from fold 7. This trend was noticeable across all folds, though the differences were not substantial enough to conclusively determine if overfitting is occurring.

This model was a big improvement over our baseline model, it achieved a significantly lower false positive rate and a much higher precision.

## Unsupervised learning

### Preprocessing for Unsupervised Learning

We made a new dataset for unsupervised learning.

In [ ]:
# Excluding Is_Bycatch
dataset_unsuper = dataset[["Art FAO Codes", "Lengdegruppe Codes", "Redskap FAO Codes", "Rundvekt", "Hovedområde start Codes"]]

We used the transformed and scaled dataset that we have used in supervised learning. We did robust scaling again to the feature ‘Rundvekt’ for the entire dataset as it was separated by a training set and a test set in the supervised learning step.

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.compose import ColumnTransformer

# Scaling only "Rundvekt" and not the others
ct = ColumnTransformer(
    [("scale", RobustScaler(), ["Rundvekt"])],
    remainder='passthrough'
)

# Fitting the scaler to the training set and not the test set to prevent data leakage
dataset_scaled = ct.fit_transform(dataset_unsuper)


####